# Chapter 02 실습: Fashion MNIST 3가지 API 구현

## 목표
Fashion MNIST를 Sequential/Functional/Subclassing 3가지 방식으로 구현하고 summary를 비교한다.

In [ ]:
import sys
sys.path.append('..')  # 상위 폴더의 utils 모듈 접근을 위한 경로 추가

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print("TensorFlow 버전:", tf.__version__)

In [ ]:
# ── Fashion MNIST 데이터 로드 및 시각화 ──────────────────────────────────
# utils.data_helpers 모듈이 있으면 사용, 없으면 직접 로드
try:
    from utils.data_helpers import load_fashion_mnist
    (x_train, y_train), (x_test, y_test) = load_fashion_mnist()
except ImportError:
    # utils 모듈이 없는 경우 Keras 내장 데이터셋 사용
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    x_train = x_train / 255.0   # [0, 255] → [0.0, 1.0] 정규화
    x_test  = x_test  / 255.0

# 클래스 이름 정의 (Fashion MNIST 10개 카테고리)
class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress',    'Coat',
    'Sandal',      'Shirt',   'Sneaker',  'Bag',       'Ankle boot'
]

print("훈련 데이터 shape:", x_train.shape)   # (60000, 28, 28)
print("테스트 데이터 shape:", x_test.shape)   # (10000, 28, 28)

# 샘플 이미지 시각화
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(x_train[i], cmap='gray')
    ax.set_title(class_names[y_train[i]], fontsize=9)
    ax.axis('off')
plt.suptitle('Fashion MNIST 샘플 이미지', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ── 모델 1: Sequential API ───────────────────────────────────────────────
seq_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='flatten'),
    tf.keras.layers.Dense(256, activation='relu', name='hidden_1'),
    tf.keras.layers.Dropout(0.3, name='dropout_1'),
    tf.keras.layers.Dense(128, activation='relu', name='hidden_2'),
    tf.keras.layers.Dropout(0.2, name='dropout_2'),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
], name='sequential_model')

print("=" * 60)
print("[모델 1] Sequential API")
print("=" * 60)
seq_model.summary()
print(f"총 파라미터: {seq_model.count_params():,}")

In [ ]:
# ── 모델 2: Functional API ───────────────────────────────────────────────
func_inputs  = tf.keras.Input(shape=(28, 28), name='image_input')
x = tf.keras.layers.Flatten(name='flatten')(func_inputs)
x = tf.keras.layers.Dense(256, activation='relu', name='hidden_1')(x)
x = tf.keras.layers.Dropout(0.3, name='dropout_1')(x)
x = tf.keras.layers.Dense(128, activation='relu', name='hidden_2')(x)
x = tf.keras.layers.Dropout(0.2, name='dropout_2')(x)
func_outputs = tf.keras.layers.Dense(10, activation='softmax', name='output')(x)

func_model = tf.keras.Model(
    inputs=func_inputs,
    outputs=func_outputs,
    name='functional_model'
)

print("=" * 60)
print("[모델 2] Functional API")
print("=" * 60)
func_model.summary()
print(f"총 파라미터: {func_model.count_params():,}")

In [ ]:
# ── 모델 3: Subclassing API ──────────────────────────────────────────────
class FashionClassifier(tf.keras.Model):
    """Fashion MNIST 분류기 — Subclassing 방식"""

    def __init__(self, num_classes=10):
        super().__init__()
        # 레이어 정의
        self.flatten  = tf.keras.layers.Flatten()
        self.dense1   = tf.keras.layers.Dense(256, activation='relu')
        self.drop1    = tf.keras.layers.Dropout(0.3)
        self.dense2   = tf.keras.layers.Dense(128, activation='relu')
        self.drop2    = tf.keras.layers.Dropout(0.2)
        self.output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

    def build(self, input_shape):
        """가중치 초기화 — 첫 번째 call 시 자동 호출됨"""
        super().build(input_shape)  # built = True 플래그 설정

    def call(self, inputs, training=False):
        """순전파 정의"""
        x = self.flatten(inputs)
        x = self.dense1(x)
        x = self.drop1(x, training=training)   # 훈련/추론 모드 전달
        x = self.dense2(x)
        x = self.drop2(x, training=training)
        return self.output_layer(x)


sub_model = FashionClassifier(num_classes=10)

# build() 호출을 위해 더미 데이터 통과
_ = sub_model(tf.zeros((1, 28, 28)))

print("=" * 60)
print("[모델 3] Subclassing API")
print("=" * 60)
sub_model.summary()
print(f"총 파라미터: {sub_model.count_params():,}")

In [ ]:
# ── 동일한 초기 가중치로 각 모델 학습 (공정한 비교) ──────────────────────

EPOCHS      = 5     # 에포크 수
BATCH_SIZE  = 128   # 미니배치 크기
SEED        = 42    # 재현성을 위한 랜덤 시드
VAL_SPLIT   = 0.1   # 검증 데이터 비율

tf.random.set_seed(SEED)  # 재현성을 위한 시드 고정

histories = {}  # 각 모델의 학습 기록 저장

for name, model in [('Sequential', seq_model),
                     ('Functional', func_model),
                     ('Subclassing', sub_model)]:
    print(f"\n{'─'*40}")
    print(f"학습 중: {name}")
    print(f"{'─'*40}")

    # 컴파일
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # 학습
    hist = model.fit(
        x_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=VAL_SPLIT,
        verbose=1
    )
    histories[name] = hist  # 기록 저장

print("\n모든 모델 학습 완료")

In [ ]:
# ── 3개 모델 검증 정확도 비교 bar chart ─────────────────────────────────

# 각 모델의 마지막 에포크 검증 정확도 추출
final_val_accs = {
    name: hist.history['val_accuracy'][-1]
    for name, hist in histories.items()
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# ─ 막대 그래프: 최종 검증 정확도 비교 ─
colors = ['#4C72B0', '#DD8452', '#55A868']  # 색상 지정
bars = ax1.bar(
    list(final_val_accs.keys()),
    list(final_val_accs.values()),
    color=colors,
    edgecolor='white',
    linewidth=1.5,
    width=0.5
)

# 막대 위에 정확도 값 표시
for bar, acc in zip(bars, final_val_accs.values()):
    ax1.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.002,
        f'{acc:.4f}',
        ha='center', va='bottom', fontsize=11, fontweight='bold'
    )

ax1.set_title('3가지 API — 최종 검증 정확도 비교', fontsize=13)
ax1.set_ylabel('검증 정확도')
ax1.set_ylim(0.8, 1.0)  # 시각적 차이를 명확하게 보기 위한 범위 설정
ax1.grid(True, alpha=0.3, axis='y')

# ─ 선 그래프: 에포크별 검증 정확도 추이 비교 ─
for (name, hist), color in zip(histories.items(), colors):
    ax2.plot(
        range(1, EPOCHS + 1),
        hist.history['val_accuracy'],
        label=name,
        color=color,
        linewidth=2,
        marker='o',
        markersize=5
    )

ax2.set_title('에포크별 검증 정확도 추이', fontsize=13)
ax2.set_xlabel('에포크')
ax2.set_ylabel('검증 정확도')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('Fashion MNIST — Sequential vs Functional vs Subclassing', fontsize=14)
plt.tight_layout()
plt.show()

# 결과 요약 출력
print("\n── 최종 검증 정확도 요약 ──")
for name, acc in final_val_accs.items():
    print(f"{name:15s}: {acc:.4f} ({acc*100:.2f}%)")

## 도전 과제

아래 항목을 직접 수정하고 실험해 보세요.

1. **BatchNormalization 추가**
   - 각 Dense 레이어 이후에 `BatchNormalization()`을 삽입하고 결과를 비교하세요.
   - `Dropout`과 `BatchNormalization`을 함께 사용할 때 순서에 주의하세요.
   ```python
   tf.keras.layers.Dense(256),
   tf.keras.layers.BatchNormalization(),  # Dense 이후, 활성화 이전
   tf.keras.layers.Activation('relu'),
   ```

2. **레이어 수 변경**
   - 은닉층을 1개로 줄였을 때와 3개로 늘렸을 때 성능을 비교하세요.
   - 파라미터 수와 정확도의 관계를 관찰하세요.

3. **학습률 튜닝**
   - `learning_rate`를 `1e-2`, `1e-3`, `1e-4`로 바꾸며 수렴 속도를 비교하세요.

4. **다른 옵티마이저 적용**
   - `Adam` 대신 `SGD(momentum=0.9)` 또는 `RMSprop`을 사용해 보세요.